## Loading Features (with generic numpy loading function) & Parcellations (with ci_lib package)

#### Imports

In [46]:
# Loading 
from pathlib import Path
from os import path
import sys
from os import listdir
from os.path import isfile, join

# Custom library from WIPAR
sys.path.append(str((Path.cwd().parent))) #use local version of ci_lib during development
import ci_lib
from ci_lib.utils import snakemake_tools
from ci_lib.features import Features, Means, Raws, Covariances, AutoCovariances, Moup, AutoCorrelations, Feature_Type
from ci_lib.plotting import graph_circle_plot, plot_glassbrain_bokeh, draw_neural_activity
from ci_lib import DecompData
from ci_lib.feature_selection import RFE_pipeline
from ci_lib.networks import construct_network, construct_network_from_feat, add_bokeh_attributes

# Processing
import numpy as np
import networkx as nx

# Dict Formating
import json     
import collections
import pandas as pd

# Plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [66]:
#Helper functions
def print_dict_keys(print_dict,level=0):
    if type(print_dict) is dict:
        print(f"Level {level}:",end=" ")
        print(list(print_dict.keys()))
        print_dict_keys(next(iter(print_dict.values())),level+1)
    else:
        print(f"Example shape of Value (feature): {print_dict.shape}")

#Used to get session name on last dim
def get_key_last_dim(rec_dict):
    #Look one level ahead
    arbitary_next_dict = next(iter(rec_dict.values()))

    if type(arbitary_next_dict) is dict:
        #If still dict continue recursion with next element
        return get_key_last_dim(arbitary_next_dict)
    else:
        #Else return a key from current level
        return list(rec_dict.keys())[0]  #TODO this will break if 'All' becomes first entry as it has no parcellation and is only the aggregation of the features from different sessions


## Loading
* Feature Values (exported, can be loaded using generic numpy loading function)
* Parcellations to get labels of nodes (custom class, load with ci_lib package)

In [68]:
''' 
Example of loading exports from WIPAR:
    Load dictionary that aggregates Feature Values {Parcellation x Feature x Condition x  (Combined Sessions + Indivdual Sessions)}
'''

#Manually define export file for loading (otherwise most recent export is loaded)
overwrite_file= None 

export_path = Path.cwd().parent/"results/exports/"

if overwrite_file is not None:
    #Loads manually defined export file
    loading_path = export_path/overwrite_file
else:
    #Iterates all export files to find most recent
    files = [export_path/f for f in listdir(export_path) if isfile(join(export_path, f))]
    try:
        loading_path = files[np.argmin([os.path.getmtime(f) for f in files])]

    except ValueError as err:
        if 'empty sequence' in str(err):
            print("Export folder is empty, run the feature rule first")
            loading_path = None 

#Loading
if loading_path is not None:   

    feat_dict = np.load(loading_path, allow_pickle=True)

    print(f"Loaded feature dict from: {loading_path}")
    print("With the following structure:")
    print_dict_keys(feat_dict)


Loaded feature dict from: /home/kuehn/WIPAR/calcium-imaging-analysis/results/exports/feats_hash6de2b164e9fc7f6457b59a40fb3c768c.npy
With the following structure:
Level 0: ["anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,MOs-L,SSp-bfd-L,VISrl-L,VISp-L']", 'anatomical_ROI~[]']
Level 1: ['mean']
Level 2: ['left', 'right']
Level 3: ['GN06.2021-03-26_10-53-05', 'GN06.2021-03-29_11-51-27', 'GN06.2021-03-01_13-57-39', 'GN06.2021-03-02_15-49-02', 'GN06.2021-03-03_14-02-06', 'GN06.2021-03-04_11-51-23', 'GN09.2021-03-26_10-13-36', 'GN09.2021-04-14_16-01-37', 'GN09.2021-04-15_11-12-30', 'GN09.2021-04-15_15-21-57', 'GN09.2021-04-16_15-11-45', 'GN09.2021-04-19_10-37-14', 'GN09.2021-04-20_13-07-10', 'GN09.2021-04-21_13-20-21', 'All']
Example shape of Value (feature): (142, 133, 8)


In [67]:
''' 
Example of loading custom classes from WIPAR:
    Loads the DecompData object for an arbirtary session for each parcellation
'''

#Get arbitrary session from last dim of feat_dict
example_session =   Path(get_key_last_dim(feat_dict)) 

#Get list of parcellations and features
parcellations = list(feat_dict.keys())

#?
features = list(next(iter(feat_dict.values())).keys())

#Get labels for each parcellation
labels = {}
for parcellation_name in list(feat_dict.keys()):

    #Load parcellation from arbitrary session to get labels
    res_path = Path.cwd().parent /'results'
    parcellation_path = Path(res_path/example_session/Path(parcellation_name)/"data.h5")
    parcellation = DecompData.load(parcellation_path)

    labels[parcellation_name] = parcellation.spatial_labels


print(labels)

#Ignore that hashes do not match, I'll remove that as hashes of dicts/objects are just not deterministic in python

{"anatomical_ROI~['VISp-R,VISrl-R,SSp-bfd-R,MOs-R,MOs-L,SSp-bfd-L,VISrl-L,VISp-L']": array(['VISpᴿ   ', 'VISrlᴿ  ', 'SSp-bfdᴿ', 'MOsᴿ    ', 'MOsᴸ    ',
       'SSp-bfdᴸ', 'VISrlᴸ  ', 'VISpᴸ   '], dtype='<U8'), 'anatomical_ROI~[]': array(['FRPᴿ    ', 'MOpᴿ    ', 'MOsᴿ    ', 'SSp-nᴿ  ', 'SSp-bfdᴿ',
       'SSp-llᴿ ', 'SSp-mᴿ  ', 'SSp-ulᴿ ', 'SSp-trᴿ ', 'SSp-unᴿ ',
       'SSsᴿ    ', 'AUDdᴿ   ', 'AUDpᴿ   ', 'AUDpoᴿ  ', 'AUDvᴿ   ',
       'VISalᴿ  ', 'VISamᴿ  ', 'VISlᴿ   ', 'VISpᴿ   ', 'VISplᴿ  ',
       'VISpmᴿ  ', 'VISliᴿ  ', 'VISporᴿ ', 'ACAdᴿ   ', 'PLᴿ     ',
       'RSPaglᴿ ', 'RSPdᴿ   ', 'RSPvᴿ   ', 'VISaᴿ   ', 'VISrlᴿ  ',
       'TEaᴿ    ', 'MOBᴿ    ', 'MOBᴸ    ', 'TEaᴸ    ', 'VISrlᴸ  ',
       'VISaᴸ   ', 'RSPvᴸ   ', 'RSPdᴸ   ', 'RSPaglᴸ ', 'PLᴸ     ',
       'ACAdᴸ   ', 'VISporᴸ ', 'VISliᴸ  ', 'VISpmᴸ  ', 'VISplᴸ  ',
       'VISpᴸ   ', 'VISlᴸ   ', 'VISamᴸ  ', 'VISalᴸ  ', 'AUDvᴸ   ',
       'AUDpoᴸ  ', 'AUDpᴸ   ', 'AUDdᴸ   ', 'SSsᴸ    ', 'SSp-unᴸ ',
       'SSp-trᴸ ', 'SSp-ulᴸ ', '